In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def getHTMLText(url,search):
    data={
        'txtKey_Words':search,
        'isQuickSearch':'true'
    }
    header={
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
        'origin':'https://www.ssrn.com',
        'referer':'https://www.ssrn.com/index.cfm/en/',
    }    
    try:
        r=requests.post(url,data=data,headers=header)
        r.raise_for_status()
        r.encoding='utf-8'
        return r.text
    except:
        return ''
    
#获取每一篇文献的名称、作者、关键字、摘要、下载数
def getEssay(lst,url,search):
    html=getHTMLText(url,search)
    soup=BeautifulSoup(html,'html.parser')
    
    for div in soup.find('div',attrs={'class':'tbody'}).find_all('div',attrs={'class':'trow'}):
        d={}
        d["文献名称"]=div.find('h3').text.strip()
    
        authors=[]
        if div.find('div',attrs={'class':'authors-list'})!=None:
            for a in div.find('div',attrs={'class':'authors-list'}).find_all('a'):
                authors.append(a.text.strip())
        d['作者']=authors
        
        if div.find('div',attrs={'class':'keywords'})!=None:
            d['关键词']=div.find('div',attrs={'class':'keywords'}).text.split(':')[1].strip()
        
        notes=div.find('div',attrs={'class':'note note-list'}).find_all('span')
        d['总页数']=notes[0].text.split(':')[1]
        
        d['发布时间']=notes[1].text.split(':')[1]
        
        #if len(notes)>2:
        #    d['上一次修改时间']=notes[2].text.split(';')[1]
        #else:
        #    d['上一次修改时间']=NULL
        if div.find('div',attrs={'class':'afiliations'})!=None:
            d['归属组织']=div.find('div',attrs={'class':'afiliations'}).text.strip()
        
        if div.find('div',attrs={'class':'downloads'}).find('span')!=None:
            d['下载数']=div.find('div',attrs={'class':'downloads'}).find_all('span')[1].text
        lst.append(d)
        
def main():
    paperInfo=[]
    searchWord=input("请输入检索词：")
    pageNum=input("请输入检索页数：")
    for i in range(int(pageNum)):
        url='https://papers.ssrn.com/sol3/results.cfm?npage='+str(i)+'&'
        getEssay(paperInfo,url,searchWord)
    df=pd.DataFrame(paperInfo)
    print(df)
    
main()